In [50]:
# Import Libraries
import warnings
warnings.filterwarnings("ignore")
import os
import glob
import matplotlib.pyplot as plt
# Keras API
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten
from keras.layers import Conv2D,MaxPooling2D,Activation,AveragePooling2D,BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
dataset_path = "datassets"

In [51]:
test_dir ="datassets\plantvillagedata\plantdisease_train_test\Test_dir"
train_dir ="datassets\plantvillagedata\plantdisease_train_test\Train_dir"

In [ ]:
import matplotlib.image as mpimg
plt.figure(figsize=(12, 5))
sp = plt.subplot(2, 5, 1)
sp.axis('Off')

img = mpimg.imread("datassets\plantvillagedata\plantdisease_train_test\Train_dir\Pepper,_bell___Bacterial_spot\image (1).JPG")
plt.imshow(img)

sp = plt.subplot(2, 5, 2)
sp.axis('Off')
img = mpimg.imread("datassets\plantvillagedata\plantdisease_train_test\Train_dir\Pepper,_bell___healthy\image (1).JPG")
plt.imshow(img)

sp = plt.subplot(2, 5, 3)
sp.axis('Off')
img = mpimg.imread("datassets\plantvillagedata\plantdisease_train_test\Train_dir\Potato___Early_blight\image (1).JPG")
plt.imshow(img)

sp = plt.subplot(2, 5, 4)
sp.axis('Off')
img = mpimg.imread("datassets\plantvillagedata\plantdisease_train_test\Train_dir\Tomato___Target_Spot\image (1).JPG")
plt.imshow(img)

sp = plt.subplot(2, 5, 5)
sp.axis('Off')
img = mpimg.imread("datassets\plantvillagedata\plantdisease_train_test\Train_dir\Tomato___Tomato_mosaic_virus\image (1).JPG")
plt.imshow(img)


sp = plt.subplot(2, 5, 6)
sp.axis('Off')
img = mpimg.imread("datassets\plantvillagedata\plantdisease_train_test\Train_dir\Tomato___Tomato_Yellow_Leaf_Curl_Virus\image (1).JPG")
plt.imshow(img)

sp = plt.subplot(2, 5, 7)
sp.axis('Off')
img = mpimg.imread("datassets\plantvillagedata\plantdisease_train_test\Train_dir\Tomato___Bacterial_spot\image (1).JPG")
plt.imshow(img)

sp = plt.subplot(2, 5, 8)
sp.axis('Off')
img = mpimg.imread("datassets\plantvillagedata\plantdisease_train_test\Train_dir\Tomato___Early_blight\image (1).JPG")
plt.imshow(img)

sp = plt.subplot(2, 5, 9)
sp.axis('Off')
img = mpimg.imread("datassets\plantvillagedata\plantdisease_train_test\Train_dir\Tomato___healthy\image (1).JPG")
plt.imshow(img)

sp = plt.subplot(2, 5, 10)
sp.axis('Off')
img = mpimg.imread("datassets\plantvillagedata\plantdisease_train_test\Train_dir\Tomato___Late_blight\image (1).jpeg")
plt.imshow(img)



In [53]:
# function to get count of images
def get_files(directory):
  if not os.path.exists(directory):
    return 0
  count=0
  for current_path,dirs,files in os.walk(directory):
    for dr in dirs:
      count+= len(glob.glob(os.path.join(current_path,dr+"/*")))
  return count

In [ ]:
train_samples =get_files(train_dir)
num_classes=len(glob.glob(train_dir+"/*"))
test_samples=get_files(test_dir) # For testing i took only few samples from unseen data. we can evaluate using validation data which is part of train data.
print(num_classes,"Classes")
print(train_samples,"Train images")
print(test_samples,"Test images")

In [55]:
# Preprocessing data. Data Augmentation
train_datagen= ImageDataGenerator(rescale=1/255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   validation_split=0.2, # validation split 20%.
                                   horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1/255)

In [ ]:
# set height and width of input image.
img_width,img_height =256,256
input_shape=(img_width,img_height,3)
#batch_size =16
batch_size =32

train_generator =train_datagen.flow_from_directory(train_dir,
                                                   target_size=(img_width,img_height),
                                                   batch_size=batch_size)
test_generator=test_datagen.flow_from_directory(test_dir,shuffle=True,
                                                   target_size=(img_width,img_height),
                                                   batch_size=batch_size)

In [ ]:
# The name of the 15 diseases.
train_generator.class_indices

In [ ]:
# CNN building.
model = Sequential()
#input_shape=(img_width,img_height,3)

#op matrix = n - f+1 = 252 = 256 - 5 +1
model.add(Conv2D(32, (5, 5),input_shape=input_shape,activation='relu'))

#252 / 3 = 84
model.add(MaxPooling2D(pool_size=(3, 3))) # 84

# 82 = 84 - 3+ 1
model.add(Conv2D(32, (3, 3),activation='relu')) # 82

#82/2 = 41
model.add(MaxPooling2D(pool_size=(2, 2))) # 41

#39 = 41 - 3+1
model.add(Conv2D(64, (3, 3),activation='relu'))

#39/2 = 19.5 ~ 19
model.add(MaxPooling2D(pool_size=(2, 2)))#19

model.add(Flatten())
#to put the data into diffrent diffrent dimention we use dense layer
#we have 15 classes so we took 15 dense layer at the end of the model
#this is best for multi class classification.
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(128,activation='relu'))

#Fully Connected Layer: we pass our flatten vector into input Layer .
#we combined these features to create a model. Finally, we have an activation
#function such as softmax or sigmoid to classify the outputs.

#Softmax is used for multi-classification in the Logistic Regression model,
#whereas Sigmoid is used for binary classification in the Logistic Regression model

model.add(Dense(num_classes,activation='softmax'))
model.summary()

In [ ]:
model_layers = [ layer.name for layer in model.layers]
print('layer name : ',model_layers)

In [ ]:
keras.utils.plot_model(model,"my_CNN_model.png")

In [ ]:
# Take one image to visualize it's changes after every layer
from keras.preprocessing import image
import numpy as np
img1 = image.load_img('datassets\plantvillagedata\plantdisease_train_test\Train_dir\Tomato___Bacterial_spot\image (1).JPG')
plt.imshow(img1)

#preprocess image
img1 = image.load_img('datassets\plantvillagedata\plantdisease_train_test\Train_dir\Tomato___Bacterial_spot\image (1).JPG', target_size=(256, 256))
img = image.img_to_array(img1)
img = img/255
img = np.expand_dims(img, axis=0)

In [ ]:
# Visualizing output after every layer.
from keras.models import Model
conv2d_6_output = Model(inputs=model.input, outputs=model.get_layer('conv2d_6').output)
max_pooling2d_6_output = Model(inputs=model.input,outputs=model.get_layer('max_pooling2d_6').output)

conv2d_7_output = Model(inputs=model.input,outputs=model.get_layer('conv2d_7').output)
max_pooling2d_7_output = Model(inputs=model.input,outputs=model.get_layer('max_pooling2d_7').output)

conv2d_8_output = Model(inputs=model.input,outputs=model.get_layer('conv2d_8').output)
max_pooling2d_8_output = Model(inputs=model.input,outputs=model.get_layer('max_pooling2d_8').output)
flatten_2_output = Model(inputs=model.input,outputs=model.get_layer('flatten_2').output)

conv2d_6_features = conv2d_6_output.predict(img)
max_pooling2d_6_features = max_pooling2d_6_output.predict(img)

conv2d_7_features = conv2d_7_output.predict(img)
max_pooling2d_7_features = max_pooling2d_7_output.predict(img)

conv2d_8_features = conv2d_8_output.predict(img)
max_pooling2d_8_features = max_pooling2d_8_output.predict(img)
flatten_2_features = flatten_2_output.predict(img)


In [ ]:
#  Visualizing output after every layer.
from keras.models import Model
conv2d_6_output = Model(inputs=model.input, outputs=model.get_layer('conv2d_6').output)
max_pooling2d_6_output = Model(inputs=model.input,outputs=model.get_layer('max_pooling2d_6').output)

conv2d_7_output = Model(inputs=model.input,outputs=model.get_layer('conv2d_7').output)
max_pooling2d_7_output = Model(inputs=model.input,outputs=model.get_layer('max_pooling2d_7').output)

conv2d_8_output = Model(inputs=model.input,outputs=model.get_layer('conv2d_8').output)
max_pooling2d_8_output = Model(inputs=model.input,outputs=model.get_layer('max_pooling2d_8').output)
flatten_2_output = Model(inputs=model.input,outputs=model.get_layer('flatten_2').output)

conv2d_6_features = conv2d_6_output.predict(img)
max_pooling2d_6_features = max_pooling2d_6_output.predict(img)

conv2d_7_features = conv2d_7_output.predict(img)
max_pooling2d_7_features = max_pooling2d_7_output.predict(img)

conv2d_8_features = conv2d_8_output.predict(img)
max_pooling2d_8_features = max_pooling2d_8_output.predict(img)
flatten_2_features = flatten_2_output.predict(img)


In [ ]:
import matplotlib.image as mpimg

fig=plt.figure(figsize=(14,7))
columns = 8
rows = 4
for i in range(columns*rows):
    #img = mpimg.imread()
    fig.add_subplot(rows, columns, i+1)
    plt.axis('off')
    plt.title('filter'+str(i))
    plt.imshow(conv2d_6_features[0, :, :, i], cmap='viridis') # Visualizing in color mode.
plt.show()


In [ ]:
for layer in model.layers :
    print(layer.name)

In [ ]:
import matplotlib.image as mpimg

fig=plt.figure(figsize=(14,7))
columns = 8
rows = 4
for i in range(columns*rows):
    #img = mpimg.imread()
    fig.add_subplot(rows, columns, i+1)
    plt.axis('off')
    plt.title('filter'+str(i))
    plt.imshow(max_pooling2d_6_features[0, :, :, i], cmap='viridis')
plt.show()

In [ ]:
for layer in model.layers :
    print(layer.name)

In [ ]:
import matplotlib.image as mpimg

fig=plt.figure(figsize=(14,7))
columns = 8
rows = 4
for i in range(columns*rows):
    #img = mpimg.imread()
    fig.add_subplot(rows, columns, i+1)
    plt.axis('off')
    plt.title('filter'+str(i))
    plt.imshow(conv2d_7_features[0, :, :, i], cmap='viridis')
plt.show()

In [ ]:
# we can also visualize in color mode.
import matplotlib.image as mpimg

fig=plt.figure(figsize=(14,7))
columns = 8
rows = 4
for i in range(columns*rows):
    #img = mpimg.imread()
    fig.add_subplot(rows, columns, i+1)
    plt.axis('off')
    plt.title('filter'+str(i))
    plt.imshow(max_pooling2d_7_features[0, :, :, i], cmap='viridis')
plt.show()

In [ ]:
import matplotlib.image as mpimg

fig=plt.figure(figsize=(16,16))
columns =8
rows = 8
for i in range(columns*rows):
    #img = mpimg.imread()
    fig.add_subplot(rows, columns, i+1)
    plt.axis('off')
    plt.title('filter'+str(i))
    plt.imshow(conv2d_8_features[0, :, :, i], cmap='viridis')
plt.show()

In [ ]:
import matplotlib.image as mpimg

fig=plt.figure(figsize=(14,14))
columns = 8
rows = 8
for i in range(columns*rows):
    #img = mpimg.imread()
    fig.add_subplot(rows, columns, i+1)
    plt.axis('off')
    plt.title('filter'+str(i))
    plt.imshow(max_pooling2d_8_features[0, :, :, i],cmap='viridis')
plt.show()

In [ ]:
# validation data.
validation_generator = train_datagen.flow_from_directory(
    train_dir, # same directory as training data
    target_size=(img_height, img_width),
    batch_size=batch_size)

In [ ]:
# Model building to get trained with parameters.
opt=keras.optimizers.Adam(lr=0.001)
model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])
train=model.fit_generator(train_generator,
                          epochs=10,
                          steps_per_epoch=train_generator.samples // batch_size,

                          validation_data=validation_generator,
                        verbose=1)



# `7.Plot For Accuracy And Losses.`

In [ ]:

from keras.callbacks import History
#history = History()

acc = train.history['accuracy']
val_acc = train.history['val_accuracy']
loss = train.history['loss']
val_loss = train.history['val_loss']

epochs = range(1, len(acc) + 1)
#Train and validation accuracy
plt.plot(epochs, acc, 'b', label='Training accurarcy')
plt.plot(epochs, val_acc, 'r', label='Validation accurarcy')
plt.title('Training and Validation accurarcy')
plt.legend()

plt.figure()
#Train and validation loss
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.legend()
plt.show()

# `8. Evaluate model using unseen data.`


In [ ]:
score,accuracy =model.evaluate(test_generator,verbose=1)
print("Test score is {}".format(score))
print("Test accuracy is {}".format(accuracy))

# `9.Saving Model.`

In [100]:
# Save entire model with optimizer, architecture, weights and training configuration.
from keras.models import load_model
model.save('crop.h5')

In [79]:
# Save model weights.
from keras.models import load_model
model.save_weights('crop_weights.h5')

In [ ]:
# Get classes of model trained on
classes = train_generator.class_indices
classes


# `10.Load Model.`

In [81]:
# Loading model and predict.
from keras.models import load_model
model=load_model('crop.h5')

Classes = ["Pepper__bell___Bacterial_spot",
           "Pepper__bell___healthy",
           "Potato___Early_blight",
           "Potato___Late_blight","Potato___healthy","Tomato_Bacterial_spot","Tomato_Early_blight",
           "Tomato_Late_blight","Tomato_Leaf_Mold","Tomato_Septoria_leaf_spot","Tomato_Spider_mites_Two_spotted_spider_mite",
           "Tomato__Target_Spot","Tomato__Tomato_YellowLeaf__Curl_Virus","Tomato__Tomato_mosaic_virus",
           "Tomato_healthy"]


# `11.Time For Predictions.`

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Pre-Processing test data same as train data.
img_width=256
img_height=256
#model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

from keras.preprocessing import image

def prepare(img_path):
    img = image.load_img(img_path, target_size=(256, 256))
    x = image.img_to_array(img)
    x = x/255
    return np.expand_dims(x, axis=0)


result = model.predict_classes([prepare('datassets\plantvillagedata\plantdisease_train_test\Train_dir\Potato___Early_blight\image (1).JPG')])
disease = image.load_img('datassets\plantvillagedata\plantdisease_train_test\Train_dir\Potato___Early_blight\image (1).JPG')
plt.imshow(disease)
print (Classes[int(result)])

In [ ]:
# Visualizing output after every layer.
from keras.models import Model
conv2d_1_output = Model(inputs=model.input, outputs=model.get_layer('conv2d_31').output)
max_pooling2d_1_output = Model(inputs=model.input,outputs=model.get_layer('max_pooling2d_9').output)

conv2d_2_output = Model(inputs=model.input,outputs=model.get_layer('conv2d_32').output)
max_pooling2d_2_output = Model(inputs=model.input,outputs=model.get_layer('max_pooling2d_10').output)

conv2d_3_output = Model(inputs=model.input,outputs=model.get_layer('conv2d_33').output)
max_pooling2d_3_output = Model(inputs=model.input,outputs=model.get_layer('max_pooling2d_11').output)
flatten_1_output = Model(inputs=model.input,outputs=model.get_layer('flatten_3').output)

conv2d_1_features = conv2d_1_output.predict(img)
max_pooling2d_1_features = max_pooling2d_1_output.predict(img)

conv2d_2_features = conv2d_2_output.predict(img)
max_pooling2d_2_features = max_pooling2d_2_output.predict(img)

conv2d_3_features = conv2d_3_output.predict(img)
max_pooling2d_3_features = max_pooling2d_3_output.predict(img)
flatten_1_features = flatten_1_output.predict(img)


In [ ]:
from keras.preprocessing import image
import numpy as np 
import matplotlib.pyplot as plt
classes = ['healthy', 'deseased']
def prepare(img_path):
    img=image.load_img(img_path,target_size=(256,256))
    x=image.img_to_array(img)
    x=x/255
    return np.expand_dims(x,axis=0)
img_path = 'datassets\plantvillagedata\plantdisease_train_test\Train_dir\Tomato___healthy\image (1).JPG'
result= model.predict(prepare(img_path))
result_class=np.argmax(result)
print("model prediction output:",result)
print("predicted class index:",result_class)
print("number of classes:",len(classes))
disease = image.load_img(img_path)
plt.imshow(disease)
plt.show()


# `12.Convert Model To "tflite format."`.
- This conversion is done because to make our model interpertable with App.
- tflite is tensorflowlite made for mobile versions.

In [106]:
import tensorflow as tf
import numpy as np
from tensorflow import lite
model=tf.keras.models.load_model('crop.h5')

converter = tf.lite.TFLiteConverter.from_keras_model('crop.h5')




In [ ]:
print('the name of the modal')